### Import Dependencies

In [ ]:
# for web scraping
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# for general use
import pandas as pd